#setup
- connect to google drive in this step

In [ ]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

In [ ]:
def get_data(individual, bodypart, h5_file):
  mouse_data = h5_file.xs(individual,level='individuals',axis=1)
  out_data = mouse_data.xs(bodypart,level='bodyparts',axis=1)
  out_data.columns = out_data.columns.droplevel("scorer")
  output = out_data.copy()
  return output

In [ ]:
def within_area(area_vector, input_coor):
  area_startx = area_vector[0]
  area_starty = area_vector[1]
  area_distx = area_vector[2]
  area_disty = area_vector[3]
  x = input_coor["x"].iloc[0]
  y = input_coor["y"].iloc[0]
  if (area_startx <= x <= (area_startx+area_distx)) and (area_starty <= y <= (area_starty+area_disty)):
    result = 1
  else:
    result = 0
  return result

In [ ]:
def euclid_dist(point1_coor, point2_coor):
  point1 = np.array((point1_coor["x"].iloc[0], point1_coor["y"].iloc[0]))
  point2 = np.array((point2_coor["x"].iloc[0], point2_coor["y"].iloc[0]))
  output_dist = np.linalg.norm(point1 - point2)
  return output_dist

#on_male_side and putative mounts by disappearing female

Note: dlc origin is in the top left; x goes to the right, y goes down

In [ ]:
for video in glob.glob(project_path + "/videos/" + "*.avi"):
# for video in [project_path + "/videos/PZ80_2.avi"]:
  video_name = os.path.splitext(os.path.basename(video))[0]

  # area_vec = [area_startx, area_starty, area_distx, area_disty]
  female_side_mat = scipy.io.loadmat(project_path + "/videos/" + video_name + "_female_side.mat")
  female_side_vec = female_side_mat['croprect'][0]
  male_side_mat = scipy.io.loadmat(project_path + "/videos/" + video_name + "_male_side.mat")
  male_side_vec = male_side_mat['croprect'][0]

  h5_file = pd.read_hdf(project_path +'/videos/'+ video_name + 'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered.h5')
  [nframes, ncols] = h5_file.shape

  mus1_snout = get_data('mus1', 'snout', h5_file)
  mus1_shoulder = get_data('mus1', 'shoulder', h5_file)
  mus1_spine1 = get_data('mus1', 'spine1', h5_file)
  mus1_spine2 = get_data('mus1', 'spine2', h5_file)
  mus1_spine3 = get_data('mus1', 'spine3', h5_file)
  mus1_spine4 = get_data('mus1', 'spine4', h5_file)
  mus1_tailbase = get_data('mus1', 'tailbase', h5_file)

  mus2_snout = get_data('mus2', 'snout', h5_file)
  mus2_shoulder = get_data('mus2', 'shoulder', h5_file)
  mus2_spine1 = get_data('mus2', 'spine1', h5_file)
  mus2_spine2 = get_data('mus2', 'spine2', h5_file)
  mus2_spine3 = get_data('mus2', 'spine3', h5_file)
  mus2_spine4 = get_data('mus2', 'spine4', h5_file)
  mus2_tailbase = get_data('mus2', 'tailbase', h5_file)

  mus3_snout = get_data('mus3', 'snout', h5_file)
  mus3_shoulder = get_data('mus3', 'shoulder', h5_file)
  mus3_spine1 = get_data('mus3', 'spine1', h5_file)
  mus3_spine2 = get_data('mus3', 'spine2', h5_file)
  mus3_spine3 = get_data('mus3', 'spine3', h5_file)
  mus3_spine4 = get_data('mus3', 'spine4', h5_file)
  mus3_tailbase = get_data('mus3', 'tailbase', h5_file)


  behavior = pd.DataFrame(index=range(nframes),columns=['behavior'])
  behavior_events = pd.DataFrame(index=range(nframes - 1), columns=['events_index', 'minutes', 'seconds', 'behavior_events'])

  for i in range(nframes):
    event_name = 'male_side'

    if within_area(female_side_vec, mus1_snout.loc[[i]]) or within_area(female_side_vec, mus1_shoulder.loc[[i]]) or within_area(female_side_vec, mus1_spine1.loc[[i]]) \
    or within_area(female_side_vec, mus1_spine2.loc[[i]]) or within_area(female_side_vec, mus1_spine3.loc[[i]]) or within_area(female_side_vec, mus1_spine4.loc[[i]]) or within_area(female_side_vec, mus1_tailbase.loc[[i]]) \
    or within_area(female_side_vec, mus2_snout.loc[[i]]) or within_area(female_side_vec, mus2_shoulder.loc[[i]]) or within_area(female_side_vec, mus2_spine1.loc[[i]]) \
    or within_area(female_side_vec, mus2_spine2.loc[[i]]) or within_area(female_side_vec, mus2_spine3.loc[[i]]) or within_area(female_side_vec, mus2_spine4.loc[[i]]) or within_area(female_side_vec, mus2_tailbase.loc[[i]]) \
    or within_area(female_side_vec, mus3_snout.loc[[i]]) or within_area(female_side_vec, mus3_shoulder.loc[[i]]) or within_area(female_side_vec, mus3_spine1.loc[[i]]) \
    or within_area(female_side_vec, mus3_spine2.loc[[i]]) or within_area(female_side_vec, mus3_spine3.loc[[i]]) or within_area(female_side_vec, mus3_spine4.loc[[i]]) or within_area(female_side_vec, mus3_tailbase.loc[[i]]):
      event_name = 'female_side'

    else:
      if not (within_area(male_side_vec, mus1_spine3.loc[[i]]) or within_area(male_side_vec, mus1_spine4.loc[[i]]) or within_area(male_side_vec, mus1_tailbase.loc[[i]])) \
      and not (within_area(male_side_vec, mus2_spine3.loc[[i]]) or within_area(male_side_vec, mus2_spine4.loc[[i]]) or within_area(male_side_vec, mus2_tailbase.loc[[i]])) \
      and (within_area(male_side_vec, mus3_snout.loc[[i]]) and within_area(male_side_vec, mus3_tailbase.loc[[i]])):
        event_name = 'mount'
      elif not (within_area(male_side_vec, mus1_spine3.loc[[i]]) or within_area(male_side_vec, mus1_spine4.loc[[i]]) or within_area(male_side_vec, mus1_tailbase.loc[[i]])) \
      and (within_area(male_side_vec, mus2_snout.loc[[i]]) and within_area(male_side_vec, mus2_tailbase.loc[[i]])) \
      and not (within_area(male_side_vec, mus3_spine3.loc[[i]]) or within_area(male_side_vec, mus3_spine4.loc[[i]]) or within_area(male_side_vec, mus3_tailbase.loc[[i]])):
        event_name = 'mount'
      elif (within_area(male_side_vec, mus1_snout.loc[[i]]) and within_area(male_side_vec, mus1_tailbase.loc[[i]])) \
      and not (within_area(male_side_vec, mus2_spine3.loc[[i]]) or within_area(male_side_vec, mus2_spine4.loc[[i]]) or within_area(male_side_vec, mus2_tailbase.loc[[i]])) \
      and not (within_area(male_side_vec, mus3_spine3.loc[[i]]) or within_area(male_side_vec, mus3_spine4.loc[[i]]) or within_area(male_side_vec, mus3_tailbase.loc[[i]])):
        event_name = 'mount'

    behavior["behavior"][i] = event_name

  events_n = 0
  for i in range(1, nframes):
    if behavior["behavior"][i] != behavior["behavior"][i-1]:
      behavior_events["events_index"][events_n] = i
      behavior_events["minutes"][events_n] = i / (30*60)
      behavior_events["seconds"][events_n] = round(i / 30) % 60
      behavior_events["behavior_events"][events_n] = behavior["behavior"][i]
      events_n = events_n + 1
  print(events_n)
  behavior_events = behavior_events.dropna(how='all')

  total_events = behavior_events.shape[0]
  for events_n in range(1, total_events - 1):
    # false positive male side due to zaps
    before_event = behavior_events["behavior_events"][events_n - 1]
    after_event = behavior_events["behavior_events"][events_n + 1]
    start_index = behavior_events["events_index"][events_n]
    end_index = behavior_events["events_index"][events_n + 1]
    current_event = behavior_events["behavior_events"][events_n]

    if end_index - start_index < 10 and before_event == after_event and current_event == 'male_side' and before_event == 'female_side':
      behavior["behavior"][start_index:end_index] = 'female_side'
      behavior_events["behavior_events"][events_n] = float('nan')
      behavior_events["behavior_events"][events_n + 1] = float('nan')

  # clean up
  behavior_events = behavior_events.dropna()
  behavior_events = behavior_events.reset_index(drop=True)
  print(behavior_events.shape)

  total_events = behavior_events.shape[0]
  for events_n in range(1, total_events - 1):\

    # mounts between two female side events or shorter than half a second
    if behavior_events["behavior_events"][events_n] == 'mount':
      beforemount_event = behavior_events["behavior_events"][events_n - 1]
      aftermount_event = behavior_events["behavior_events"][events_n + 1]
      mount_start_index = behavior_events["events_index"][events_n]
      mount_end_index = behavior_events["events_index"][events_n + 1]

      if beforemount_event == 'female_side' and aftermount_event == 'female_side':
        behavior["behavior"][mount_start_index:mount_end_index] = 'female_side'
        behavior_events["behavior_events"][events_n] = float('nan')
        if beforemount_event == 'female_side' and aftermount_event == 'female_side':
          behavior_events["behavior_events"][events_n + 1] = float('nan')
      elif mount_end_index - mount_start_index < 15:
        behavior["behavior"][mount_start_index:mount_end_index] = beforemount_event
        behavior_events["behavior_events"][events_n] = float('nan')
        behavior_events["behavior_events"][events_n + 1] = float('nan')

  # clean up
  behavior_events = behavior_events.dropna()
  behavior_events = behavior_events.reset_index(drop=True)

  with open(output_dir + video_name + '_behavior.pickle', 'wb') as f:
      pickle.dump(behavior, f)
  with open(output_dir + video_name + '_behavior_events.pickle', 'wb') as f:
      pickle.dump(behavior_events, f)

1484
(1332, 4)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Behavior categorization logic:
- if mouse detected on female side, -> definitely female_side

- if no mouse detected on female side, -> putative male_side
 - problem: when grooming, sometimes does not detect lone female on female side

- if no mouse detected on female side, only 1 mouse back detected on male side, and both adjacent events were female_side, -> likely undetected female_side

- if no mouse detected on female side, only 1 mouse back detected on male side, and at least 1 adjacent event was not female_side -> putative mount
 - problem: sometimes putative mounts are too quick to really be mounts

- if putative mount and lasts more than 1 second or 30 frames, -> likely mount


at this point:
- 'female_side' means mouse detected on female side
- 'male_side' means not 1 (so 0 or 2) mouse back detected on male side and no mice on female side
- 'mount' means 1 mouse back detected on male side and no mice on female side

#make labelled behavior video

In [ ]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14/'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

end_frame = 36000
# for video in [project_path + "/videos/PZ80_2.avi"]:
for video in glob.glob(project_path + "/videos/" + "*.avi"):
  video_name = os.path.splitext(os.path.basename(video))[0]
  
  labeled_video = video_name+'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered_id_labeled.mp4'

  behavior_file = open(output_dir + video_name + '_behavior.pickle', 'rb')
  behavior = pickle.load(behavior_file)
  behavior_file.close()

  behavior_events_file = open(output_dir + video_name + '_behavior_events.pickle', 'rb')
  behavior_events = pickle.load(behavior_events_file)
  behavior_events_file.close()

  cap = cv2.VideoCapture(project_path + 'videos/' + labeled_video)

  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")

  while cap.isOpened():
    ret,frame = cap.read()
    print(frame.shape)
    frame_width = int(frame.shape[1])
    frame_height = int(frame.shape[0])
    break

  output_video = output_dir + video_name + '_behavior_video.mp4'

  out = cv2.VideoWriter(output_video,cv2.VideoWriter_fourcc(*'MP4V'), 30, (frame_width,frame_height))

  count = 0
  while cap.isOpened():
    ret,frame = cap.read()
    current_behavior = behavior["behavior"][count]
    text_color = (0,0,0)
    if current_behavior == 'male_side':
      text_color = (0, 0, 150)
    elif current_behavior == 'mount':
      text_color = (0, 0, 255)
    elif current_behavior == 'female_side':
      text_color = (50, 0, 0)
    else:
      print(current_behavior)
      print(count)
    cv2.putText(frame, str(current_behavior), (25, 25), cv2.FONT_HERSHEY_DUPLEX, 1, text_color, 2)
    cv2.putText(frame, str(count), (25, frame_height), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2)
    out.write(frame)

    count = count + 1
    if count > end_frame:
      break
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  cap.release()
  out.release()

(166, 286, 3)
nan
392
nan
393
nan
394
nan
395
nan
396
nan
397
nan
398
nan
632
nan
696
nan
697
nan
704
nan
705
nan
706
nan
707
nan
708
nan
709
nan
710
nan
711
nan
712
nan
713
nan
714
nan
715
nan
716
nan
718
nan
719
nan
1901
nan
1913
nan
1961
nan
1962
nan
1967
nan
1970
nan
2018
nan
2019
nan
2020
nan
2021
nan
2094
nan
3603
nan
3604
nan
3605
nan
3606
nan
3734
nan
3735
nan
3736
nan
3737
nan
3738
nan
3739
nan
3740
nan
3741
nan
3742
nan
3743
nan
3744
nan
3745
nan
3746
nan
3747
nan
3749
nan
3750
nan
3751
nan
3752
nan
3753
nan
3754
nan
3755
nan
3756
nan
3757
nan
4259
nan
4260
nan
4261
nan
4456
nan
4457
nan
4458
nan
4459
nan
4460
nan
4461
nan
4462
nan
4463
nan
4464
nan
4465
nan
4466
nan
4714
nan
4715
nan
4716
nan
4717
nan
4721
nan
4722
nan
4723
nan
4726
nan
4727
nan
4735
nan
4740
nan
4741
nan
4742
nan
4744
nan
4758
nan
4759
nan
4760
nan
4764
nan
4765
nan
4774
nan
4775
nan
4778
nan
4779
nan
4783
nan
4784
nan
4785
nan
4787
nan
4788
nan
4789
nan
4792
nan
4793
nan
4794
nan
4795
nan
4797
nan
4800
nan

#read previous behavior data pickles and edit behavior events

In [ ]:
video_name = 'PZ80_2'

behavior_file = open(output_dir + video_name + '_behavior.pickle', 'rb')
behavior = pickle.load(behavior_file)
behavior_file.close()

behavior_events_file = open(output_dir + video_name + '_behavior_events.pickle', 'rb')
behavior_events = pickle.load(behavior_events_file)
behavior_events_file.close()

print(video_name)
first_20_min = behavior.truncate(after=(36000 - 1)).copy()
male_side_seconds = (first_20_min[first_20_min["behavior"] == 'male_side']["behavior"].count() + first_20_min[first_20_min["behavior"] == 'mount']["behavior"].count()) / 30
male_side_percent = male_side_seconds / (36000 / 30)
mount_seconds = first_20_min[first_20_min["behavior"] == 'mount']["behavior"].count() / 30

first_20_min_events = behavior_events.copy()
male_side_entrances = first_20_min_events[first_20_min_events["behavior_events"] == 'male_side']["behavior_events"].count()
mount_events = first_20_min_events[first_20_min_events["behavior_events"] == 'mount']["behavior_events"].count()

print('male_side seconds: ', male_side_seconds)
print('male_side percent: ', male_side_percent)
print('mount seconds: ', mount_seconds)
print('mount events: ', mount_events)

PZ80_2
male_side seconds:  418.43333333333334
male_side percent:  0.3486944444444445
mount seconds:  17.966666666666665
mount events:  15


In [ ]:
print(first_20_min_events[first_20_min_events["behavior_events"] == 'mount'])

    events_index    minutes seconds behavior_events
1            583   0.323889      19           mount
3           1365   0.758333      46           mount
67         11516   6.397778      24           mount
94         13085   7.269444      16           mount
96         13160   7.311111      19           mount
98         13266       7.37      22           mount
125        14778       8.21      13           mount
127        14868       8.26      16           mount
129        15019   8.343889      21           mount
160        21250  11.805556      48           mount
224        31615  17.563889      34           mount
226        33563  18.646111      39           mount
233        36671  20.372778      22           mount
235        37245  20.691667      42           mount
237        37724  20.957778      57           mount


In [ ]:
def change_behavior_event(behavior, behavior_events, event_index, new_event):
  behavior_events_i = behavior_events.loc[behavior_events['events_index'] == event_index]
  behavior_events["behavior_events"][behavior_events_i] = new_event
  start_index = behavior_events["events_index"][behavior_events_i]
  end_index = behavior_events["events_index"][behavior_events_i + 1]
  behavior["behavior"][start_index:end_index]

In [ ]:
# event_index = 1
# new_event = 'female_side'
# change_behavior_event(behavior, behavior_events, new_event)